In [5]:
import React from 'react'

const F = ({name}) => <>{name}</>
const G = ({name, children=null}) => <>{name}.{children}</>
const f = <F name="name" />
const g = <G name="g"><F name="name" /></G>
g

{
  '$$typeof': Symbol(react.element),
  type: [Function: G],
  key: null,
  ref: null,
  props: {
    name: 'g',
    children: {
      '$$typeof': Symbol(react.element),
      type: [Function: F],
      key: null,
      ref: null,
      props: [Object],
      _owner: null,
      _store: {}
    }
  },
  _owner: null,
  _store: {}
}


In [21]:
import React from 'react'

const A = ({name="A", children=null}) => <>{name}{"["}{children}{"]"}</>
const a = <A><A/></A>
a

{
  '$$typeof': Symbol(react.element),
  type: [Function: A],
  key: null,
  ref: null,
  props: {
    children: {
      '$$typeof': Symbol(react.element),
      type: [Function: A],
      key: null,
      ref: null,
      props: {},
      _owner: null,
      _store: {}
    }
  },
  _owner: null,
  _store: {}
}


In [59]:
import React from 'react'

const A = ({name="A", children=null}) => <>{name}{"["}{children}{"]"}</>
const a = <A><A/></A>
const defitem = {}
defitem.type=null
defitem.props={}
const resolveComponentInstance = (item={type: null, props: {}}) => {
    if (!(item?.type)) {
        return item
    }
    const {type, props} = item
    const Component = type
    const {children, ...newprops} = props
    if (children) {
        if (Array.isArray(children)) {
            const achildren = children.map(c => resolveComponentInstance(c))
            return <Component {...newprops}>{achildren}</Component>
        } else {
            return <Component {...newprops}>{resolveComponentInstance(children)}</Component>
        }
    } else {
        return <Component {...newprops} />
    }   
}


const aa = resolveComponentInstance(a)
aa

{
  '$$typeof': Symbol(react.element),
  type: [Function: A],
  key: null,
  ref: null,
  props: {
    children: {
      '$$typeof': Symbol(react.element),
      type: [Function: A],
      key: null,
      ref: null,
      props: {},
      _owner: null,
      _store: {}
    }
  },
  _owner: null,
  _store: {}
}


In [74]:
import React from 'react'
const Outlet = () => null

const resolveComponent = (item={type: null, props: {}}) => {
    let result = null
    if (!(item?.type)) {
        result = () => item
    }
    const {type, props} = item
    const Component = type
    if (Component === Outlet) {
        result = ({children=null}) => <>{children}</>
    }
    const {children, ...newprops} = props
    if (children) {
        if (Array.isArray(children)) {
            const achildren = children.map(c => resolveComponent(c))
            result <Component {...newprops}>{achildren}</Component>
        } else {
            return <Component {...newprops}>{resolveComponent(children)}</Component>
        }
    } else {
        return <Component {...newprops} />
    }   
}
    
const resolveComponentInstance = (item={type: null, props: {}}) => {
    if (!(item?.type)) {
        return () => item
    }
    
    const {type, props} = item
    const Component = type
    const {children, ...newprops} = props
    const fixedchildren = children
    const Result = ({children=null, ...props}) => {
        return <Component {...newprops} {...props}>{fixedchildren}{children}</Component>
    }
    return Result
}

const A = ({name="A", children=null}) => <>{name}.{"["}{children}{"]"}</>
const a = <A><A/></A>
const BA = resolveComponentInstance(a)
const ba = <BA name="BA"/>
ba
DisplayReact(ba)

BA<!-- -->.<!-- -->[<!-- -->A<!-- -->.<!-- -->[<!-- -->]<!-- -->]

In [79]:
import React from 'react'

const MappedComponent = ({Component=<>undef</>, ...propertyMappers}) => {
    const constantProperties = {}
    const functionalProperties = {}
    for(const key in propertyMappers) {
        /**
         * prochazime vsechny polozky v propertyMappers a na zaklade nazvu klice vytvarime novou property
         * jeji nazev je nazvem klice, jeji hodnota je dana volanim mapovaci funkce
         * parametrem mapovaci funkce jsou vsechny predane properties s vyjimkou children
         * 
         * pokud v mapovani neni funkce, povazuje se to za konstantu a tato se primo preda
         */
        let value = propertyMappers[key]
        if (typeof value === 'function') {
            functionalProperties[key] = value
        } else {
            constantProperties[key] = value
        }
    }
    const CClass = resolveComponentInstance(Component)
    const Result = ({children=null, ...props}) => {
        const properties = {...constantProperties}
        /**
         * vezmou se properties, ktere jsou definovany jako konstanty
         */

        /**
         * pridaji se dynamicke properties (definovane pomoci funkci)
         */
        for(const key in functionalProperties) {
            properties[key] = functionalProperties[key](props)
        }

        /**
         * properties jsou definovany, ted jen zavolame komponentu
         */
        //return <Component {...properties}>{children}</Component>
        return <CClass {...properties}>{children}</CClass>
    }
    Result.displayName = Component.displayName || "MC";
    return Result
}

const A = ({children=null, ...props}) => {
    return <div>A: {JSON.stringify(props)} / {children}</div>
}
const a = <A />
a
const M = MappedComponent({Component: (<A><A name="inner"/></A>), name: "M", value: props => props?.value})
//const M = MappedComponent({Component: <A />, name: "M", value: props => props?.value})
const m = <M value={"value"}/>
m
DisplayReact(m)

A: <!-- -->{"name":"M","value":"value"}<!-- --> / A: <!-- -->{"name":"inner"}<!-- --> /

unknown msg_type: comm_open
unknown msg_type: comm_msg
unknown msg_type: comm_open
unknown msg_type: comm_msg


In [47]:
import React from 'react'
import ReactDOMServer from 'react-dom/server';
import * as tslab from 'tslab';

function DisplayReact(element) {
    const html = ReactDOMServer.renderToString(element);
    tslab.display.html(html);
}

const A = ({...props}) => {
    return <div>{JSON.stringify(props)}</div>
}
const M = MappedComponent({Component: A, name: "M", value: props => props?.value})
const m = <M value={"value"}/>
m

DisplayReact(m)

{"name":"M","value":"value","children":null}

In [56]:
import React from 'react'
import ReactDOMServer from 'react-dom/server';
import * as tslab from 'tslab';

const createUI = (dataPattern={}, definition=[], Component=<>undef</>) => {
    const results = []
    const indexDefinition =  definition.find(d => !d?.path)
    for(const key in dataPattern){
        const datadefinition = definition.find(d => d?.path === key)
        if (!datadefinition) { continue }
        results.push(MappedComponent({"Component": datadefinition.element, ...datadefinition?.properties}))
    }
    console.log("results", results)
    const Encapsulator = MappedComponent({Component: indexDefinition.Component, ...indexDefinition.properties})
    const Result = ({children=null, ...props}) => {
        //const ChildrenFromResults = results.map(
        //    C => <C {...props} />
        //)
        //return <Encapsulator {...props}>{ChildrenFromResults}{children}</Encapsulator>
        return <Encapsulator {...props}>{JSON.stringify(props)}</Encapsulator>
    }
    Result.displayName = Component.displayName || "UI";
    return Result
}

const A = ({children=null, ...props}) => {
    return <div>{JSON.stringify(props)}<br />{children}</div>
} 
const User = A
const dp = {
    user: {
        name: "username",
        group: {
            name: "groupname"
        }
    }
}
const dd = [
    {element: A, properties: {theme: "a", user: data => data?.user || "user not found"}},
    {path: "user", element: <User />, properties: {theme: "group", group: data => data?.group || "group not found"}}
]

const UI = createUI(dp, dd, A)
console.log("UI", UI)
const ui = <UI user={dp.user} />
DisplayReact(ui)


results [ [Function: Result] { displayName: 'MC' } ]
UI [Function: Result] { displayName: 'UI' }


    at Result (evalmachine.MC:27:23)
    at Result (evalmachine.UI:47:23)
/home/jovyan/node_modules/react-dom/cjs/react-dom-server-legacy.node.development.js:6984
    throw fatalError;
    ^

Error: Element type is invalid: expected a string (for built-in components) or a class/function (for composite components) but got: object.
    at renderElement (/home/jovyan/node_modules/react-dom/cjs/react-dom-server-legacy.node.development.js:6047:9)
    at renderNodeDestructiveImpl (/home/jovyan/node_modules/react-dom/cjs/react-dom-server-legacy.node.development.js:6108:11)
    at renderNodeDestructive (/home/jovyan/node_modules/react-dom/cjs/react-dom-server-legacy.node.development.js:6080:14)
    at renderIndeterminateComponent (/home/jovyan/node_modules/react-dom/cjs/react-dom-server-legacy.node.development.js:5789:7)
    at renderElement (/home/jovyan/node_modules/react-dom/cjs/react-dom-server-legacy.node.development.js:5950:7)
    at renderNodeDestructiveImpl (/home/jovyan/node_modules/r

1:1 - Circular definition of import alias 'os'.
1:1 - 'import ... =' can only be used in TypeScript files.
3:1 - '=' expected.
3:9 - ';' expected.
3:10 - Expression expected.
unknown msg_type: comm_open
unknown msg_type: comm_msg
